In [ ]:
%%capture
!pip install unsloth
!pip install bitsandbytes
!pip install gradio
!pip install git+https://github.com/openai/whisper.git
!pip install ffmpeg

In [ ]:
dtype = None

In [ ]:
import re
import whisper
from transformers import AutoTokenizer, TextStreamer, TextIteratorStreamer
from threading import Thread
from unsloth import FastLanguageModel

# Whisper for audio transcription
whisper_model = whisper.load_model("base")

# Large Language Models for text generation
models = [
    {
        "name": "Llama 3B 😁",
        "path": "martinbravo/llama_finetuned_test",
        "max_seq_length": 2048,
    },
    {
        "name": "Llama 1B - Finetuned w/ Finetome 🚀",
        "path": "desireeBill/lora_model",
        "max_seq_length": 2048,
    },
    {
        "name": "Llama + Emojis 🐼",
        "path": "desireeBill/emoji_model",
        "max_seq_length": 2048,
    }
]

# Initialize all the models
for model_id in range(len(models)):

    # Load model and tokenizer
    selected_model = models[model_id]
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=selected_model["path"],
        max_seq_length=selected_model["max_seq_length"],
        dtype=dtype,
        load_in_4bit=True,
    )
    FastLanguageModel.for_inference(model)
    selected_model["model"] = model
    selected_model["tokenizer"] = tokenizer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 168MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

...

Unsloth 2024.12.4 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


adapter_model.safetensors:   0%|          | 0.00/22.6M [00:00<?, ?B/s]

==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


adapter_model.safetensors:   0%|          | 0.00/22.6M [00:00<?, ?B/s]

In [ ]:
def generate_response(messages, model_id):

    selected_model = models[model_id]
    model = selected_model["model"]
    tokenizer = selected_model["tokenizer"]

    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
        padding=True,
    ).to("cuda")

    # Generate the response
    streamer = TextIteratorStreamer(tokenizer, skip_prompt=True)
    generation_kwargs = {"input_ids": inputs, "streamer": streamer, "max_new_tokens": 1500}
    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()

    # Collect the response
    response = ""
    for new_text in streamer:
        new_text = new_text.replace("<|eot_id|>", "")
        response += new_text

    return response

In [ ]:
import gradio as gr
import time

# Whisper for transcription
def transcribe_audio(file):
    if file is None:
        return "Please record or upload an audio file."
    result = whisper_model.transcribe(file)
    return result["text"]

# Handle user input (text and audio)
def add_message(history, text_input, audio_input):

    message = ""
    # Handle text input
    if text_input:
        message += text_input

    # Handle audio input
    if audio_input is not None:
        transcription = transcribe_audio(audio_input)
        message += f"\n{transcription}"

    history.append({"role": "user", "content": message})

    return history, "", None

# Generate response based on selected model
def chatbot_call(history, model_id):
    messages = [{"role": h["role"], "content": h["content"]} for h in history]
    response = generate_response(messages, model_id)
    return response


# Stream bot response
def bot(history: list, model_id):
    response = chatbot_call(history, model_id)
    history.append({"role": "assistant", "content": ""})
    for character in response:
        character = character.replace("<|eot_id|>", "")
        history[-1]["content"] += character
        time.sleep(0.05)  # Simulate streaming response
        yield history


In [ ]:
# Gradio interface
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(elem_id="chatbot", bubble_full_width=False, type="messages")

    with gr.Row():
        chat_input = gr.Textbox(
            interactive=True,
            placeholder="Enter message...",
            show_label=False,
            lines=1,
        )
        mic_input = gr.Audio(sources="microphone", type="filepath", label="Speak Here")
        model_selector = gr.Dropdown(
            choices=[(m["name"], str(i)) for i, m in enumerate(models)],
            type="index",
            label="Select Model",
        )

        # Button to submit input
        send_btn = gr.Button("Send")

        # Chat handling
        send_btn.click(
            add_message,
            [chatbot, chat_input, mic_input],
            [chatbot, chat_input, mic_input],
        ).then(
            bot, [chatbot, model_selector], chatbot
        )

In [ ]:
demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://8dc976a08f2fdb8665.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
